# PART 2: SQLite #

Export the country club data from PHPMyAdmin, and connect to a local SQLite instance from Jupyter notebook 
for the following questions.  

QUESTIONS:
#### Q10: Produce a list of facilities with a total revenue less than 1000.
The output of facility name and total revenue, sorted by revenue. Remember
that there's a different cost for guests and members! */

#### Q11: Produce a report of members and who recommended them in alphabetic surname,firstname order */


#### Q12: Find the facilities with their usage by member, but not guests */


#### Q13: Find the facilities usage by month, but not guests */

______________________________

#### Q10: Produce a list of facilities with a total revenue less than 1000.
The output of facility name and total revenue, sorted by revenue. Remember
that there's a different cost for guests and members! */

In [1]:
from sqlalchemy import create_engine
import pandas as pd

In [2]:
# this function will get the database url (`db`) and the query (`q`) as parameter, 
# and return the result of the query as a pandas DataFrame

def run_query(db,q):
    engine = create_engine(db)
    with engine.connect() as con:
        rs = con.execute(q)
        df = pd.DataFrame(rs.fetchall())
        df.columns = rs.keys()
    return df

In [3]:
db = 'sqlite:///sqlite_db_pythonsqlite.db'

In [4]:
q = """
SELECT  f.facid, f.name,
        SUM( CASE WHEN b.memid = 0 THEN f.guestcost * b.slots
             ELSE f.membercost * b.slots END) AS revenue

FROM    facilities AS f
        LEFT JOIN bookings AS b USING (facid)
        LEFT JOIN members AS m USING (memid)

GROUP BY f.facid
HAVING revenue < 1000
ORDER BY revenue
"""

In [5]:
run_query(db,q)

,facid,name,revenue
0,3,Table Tennis,180
1,7,Snooker Table,240
2,8,Pool Table,270


To get a more accurate answer the `monthlymaintenance` column must me taken in consideration. Since the cost is in monthly basis, the month should be extracted from the `starttime` column and the data grouped by the month.

In [6]:
q ="""
SELECT  f.facid, f.name, 
        strftime('%m', b.starttime) AS month,
    
        SUM(CASE WHEN b.memid = 0 THEN f.guestcost * b.slots
                  ELSE f.membercost * b.slots END) AS gross,
        
        f.monthlymaintenance AS maintenance,
        (SUM(CASE WHEN b.memid = 0 THEN f.guestcost * b.slots
                  ELSE f.membercost * b.slots END)) - f.monthlymaintenance  AS revenue

FROM    facilities as f 
        LEFT JOIN bookings AS b USING(facid) 
        LEFT JOIN members AS m USING (memid)
GROUP BY f.facid, Month
HAVING revenue < 1000
ORDER BY revenue
"""

The revenue for each facility is calculated by utilizing a <u>CASE WHEN</u> expression to determine the guest or the member cost, and using `LEFT JOINs` to merge the tables. Grouping the data by facility enables the CASE statement to be encapsulated within a `SUM()` function to calculate the revenue. Extracting the month from the `starttime`column and grouping by month enables to deduct the `monthlymaintenance` column from the GROSS revenue, ensuring an accurate revenue calculation. 
Finally, the results are filtered and ordered by  therevenue

In [7]:
run_query(db,q)

,facid,name,month,gross,maintenance,revenue
0,5,Massage Room 2,07,1359.2,3000,-1640.8
1,3,Table Tennis,07,30.0,10,20.0
2,8,Pool Table,07,35.0,15,20.0
3,3,Table Tennis,08,40.0,10,30.0
4,7,Snooker Table,08,50.0,15,35.0
5,7,Snooker Table,07,80.0,15,65.0
6,8,Pool Table,08,95.0,15,80.0
7,7,Snooker Table,09,110.0,15,95.0
8,3,Table Tennis,09,110.0,10,100.0
9,8,Pool Table,09,140.0,15,125.0


Apperantly the dataset is composed for 3 months (July, August and September).<br>The **Total Quarterly Net Revenue** can be calculated:

In [8]:
# confirming that the data are only for Q3:
run_query(db, "SELECT DISTINCT strftime('%m',starttime) AS month FROM bookings")

,month
0,07
1,08
2,09


In [9]:
q = """
SELECT  f.facid, f.name,
        SUM( CASE WHEN b.memid = 0 THEN f.guestcost * b.slots
             ELSE f.membercost * b.slots END) - (3 * f.monthlymaintenance) AS revenue

FROM    facilities AS f
        LEFT JOIN bookings AS b USING (facid)
        LEFT JOIN members AS m USING (memid)

GROUP BY f.facid
HAVING revenue < 1000
ORDER BY revenue
"""

In [10]:
run_query(db,q)

,facid,name,revenue
0,3,Table Tennis,150
1,7,Snooker Table,195
2,8,Pool Table,225


_____

The facilities having the revenue less than 1000 for the **Q3** are <u>Table Tennis</u> id=3,  <u>Snooker Table</u>  id = 7 and <u>Pool Table</u> id =8

_____________

#### Q11: Produce a report of members and who recommended them in alphabetic surname, firstname order


In [11]:
q = """
SELECT  m1.memid, m1.firstname, m1.surname, m1.joindate, 
        (m2.firstname||' '|| m2.surname) AS recommendedby

FROM    members AS m1 LEFT JOIN members AS m2
        ON m1.recommendedby = m2.memid

ORDER BY m1.surname, m1.firstname"""

run_query(db,q)

,memid,firstname,surname,joindate,recommendedby
0,15,Florence,Bader,2012-08-10 17:52:03,Ponder Stibbons
1,12,Anne,Baker,2012-08-10 14:23:22,Ponder Stibbons
2,16,Timothy,Baker,2012-08-15 10:34:25,Jemima Farrell
3,8,Tim,Boothe,2012-07-25 16:02:35,Tim Rownam
4,5,Gerald,Butters,2012-07-09 10:44:09,Darren Smith
5,22,Joan,Coplin,2012-08-29 08:32:41,Timothy Baker
6,36,Erica,Crumpet,2012-09-22 08:36:38,Tracy Smith
7,7,Nancy,Dare,2012-07-25 08:59:12,Janice Joplette
8,28,David,Farrell,2012-09-15 08:22:05,None
9,13,Jemima,Farrell,2012-08-10 14:28:01,None


________

#### Q12: Find the facilities with their usage by member, but not guests */

In [12]:
q = """

SELECT  f.facid, f.name, COUNT(b.bookid) AS usage_count

FROM    facilities AS f 
            INNER JOIN bookings AS b USING(facid)
            INNER JOIN members AS m USING(memid)
WHERE m.memid != 0
GROUP BY f.facid;
"""

In [13]:
run_query(db,q)

,facid,name,usage_count
0,0,Tennis Court 1,308
1,1,Tennis Court 2,276
2,2,Badminton Court,344
3,3,Table Tennis,385
4,4,Massage Room 1,421
5,5,Massage Room 2,27
6,6,Squash Court,195
7,7,Snooker Table,421
8,8,Pool Table,783


___

#### Q13: Find the facilities usage by month, but not guests */

In [14]:
q = """

SELECT  f.facid, f.name,
        strftime('%m', b.starttime) AS month, 
        COUNT(b.bookid) AS usage_count

FROM    facilities AS f 
            INNER JOIN bookings AS b USING(facid)
            INNER JOIN members AS m USING(memid)
WHERE m.memid != 0
GROUP BY month,f.facid;
"""

In [15]:
run_query(db, q)

,facid,name,month,usage_count
0,0,Tennis Court 1,07,65
1,1,Tennis Court 2,07,41
2,2,Badminton Court,07,51
3,3,Table Tennis,07,48
4,4,Massage Room 1,07,77
5,5,Massage Room 2,07,4
6,6,Squash Court,07,23
7,7,Snooker Table,07,68
8,8,Pool Table,07,103
9,0,Tennis Court 1,08,111
